In [1]:
%config IPCompleter.greedy=True

In [10]:
from pprint import pprint
import folium
import geopy.distance
import math
import numpy as np
from IPython import embed

In [3]:
import ee
ee.Authenticate()

Enter verification code:  4/2AHZNSPRGfisvYtL1aE9RPvziMYo5IaSUa7T50RyS3phitFpYWt63ng



Successfully saved authorization token.


In [4]:
ee.Initialize()

In [5]:
# constants for each satellite
ls5 = {
    'imagecollection_id' : 'LANDSAT/LT05/C01/T1_SR',
    'pixel_size' : 30,
    'bands' : ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'],
    'time_field' : 'system:time_start'
}
ls8 = {
    'imagecollection_id' : 'LANDSAT/LC08/C01/T1_SR',
    'pixel_size' : 30,
    'bands' : ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'],
    'time_field' : 'system:time_start'
}
s2 = {
    'imagecollection_id' : 'COPERNICUS/S2_SR',
    'pixel_size' : 20,
    'bands' : ['B2', 'B3', 'B4', 'B8A', 'B11', 'B12', 'MSK_CLDPRB', 'QA60'], #check up msk and qa60 msk is 20m qa60 is 60m
    'time_field' : 'system:time_start'
}
modis = {
    'imagecollection_id' : 'MODIS/006/MCD43A4',
    'pixel_size' : 500,
    'bands' : ['Nadir_Reflectance_Band3', 'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band6', 'Nadir_Reflectance_Band7'],
    'time_field' : 'system:time_start'
}
image_sets = {
    'ls5' : ls5,
    'ls8' : ls8,
    's2' : s2,
    'modis' : modis
}

In [6]:
# User inputted data
west_long = 133.84410607205484
east_long = west_long+(134.7422383962736 - west_long)/5 #temporary just to make a smaller area
north_lat = -22.638743081339985
south_lat = north_lat - (north_lat - (-23.52560283638598))/5 #temporary just to make a smaller area
date_range = ('2014-08-01', '2014-12-31')
prediction_dates = ['2014-08-25', '2014-09-21']
satellite_choice = 'ls8'
overlap_fraction = 1 # how much of the minimum block size to add for the overlap. Default is one.
percent_clear = 95

# calculated baseline variables from user data
corner_coords = [[east_long, south_lat], [west_long, south_lat], [west_long, north_lat], [east_long, north_lat], [east_long, south_lat]]
block_size = math.floor(image_sets['modis']['pixel_size']/image_sets[satellite_choice]['pixel_size'])
# minimum tile size is 10 MODIS pixels, this following formula just accounts for how many pixels we need at minimum to have it as a multiple of block size which is a PSRFM req
min_tile_dim_px = block_size*10
min_tile_dim_km = (min_tile_dim_px * image_sets[satellite_choice]['pixel_size'])/1000
# min_tile_dim_km, east_long, block_size, west_long, east_long, north_lat, south_lat

In [7]:
# Creating the geometry for tiles within the specified coordinates
# first calculate the side lengths of the region selected
x_dist = geopy.distance.geodesic([south_lat, east_long], [south_lat, west_long]).km
y_dist = geopy.distance.geodesic([north_lat, east_long], [south_lat, east_long]).km

# determine the number of tile segments to fully cover the region, rounded up to ensure overlap
# need to look at this later to ensure that each tile has enough of an overlap to ensure pixels are a multiple of block size (aka cropping)
x_tile_segments = math.floor(x_dist/min_tile_dim_km)
y_tile_segments = math.floor(y_dist/min_tile_dim_km)

# generate a list of ordered coordinates (west to east, north to south) based on the number of tiles
# creating an overlap of approx 1 block_size pixels to ensure that each tile after cropping for PSRFM will still retain some overlap
km_long = abs(east_long - west_long)/x_dist
km_lat = abs(north_lat - south_lat)/y_dist
long_overlap = ((overlap_fraction * block_size * image_sets[satellite_choice]['pixel_size'])/1000) * km_long
lat_overlap = ((overlap_fraction * block_size * image_sets[satellite_choice]['pixel_size'])/1000) * km_lat

# determining the coordinate jumps for each tile(sans overlap)
x_coord_increment = abs(east_long - west_long)/x_tile_segments + long_overlap
y_coord_increment = abs(north_lat - south_lat)/y_tile_segments + lat_overlap

# creating the lists
west_tile_coords = [east_long - (tile_no + 1) * x_coord_increment for tile_no in reversed(range(x_tile_segments))]
east_tile_coords = [west_long + (tile_no + 1) * x_coord_increment for tile_no in range(x_tile_segments)]

north_tile_coords = [south_lat + (tile_no + 1) * y_coord_increment for tile_no in reversed(range(y_tile_segments))]
south_tile_coords = [north_lat - (tile_no + 1) * y_coord_increment for tile_no in range(y_tile_segments)]

tiles = np.empty((x_tile_segments, y_tile_segments), ee.Geometry)
for col in range(x_tile_segments):
    east_coord = east_tile_coords[col]
    west_coord = west_tile_coords[col]
    for row in range(y_tile_segments):
        north_coord = north_tile_coords[row]
        south_coord = south_tile_coords[row]
        tiles[col, row] = ee.Geometry.Rectangle([west_coord, south_coord, east_coord, north_coord])
        
# x_dist, y_dist, min_tile_dim_km, x_tile_segments, y_tile_segments, x_coord_increment, y_coord_increment, west_tile_coords, east_tile_coords, north_tile_coords, south_tile_coords, tiles

In [24]:
# generating arrays of the fine and coarse res imagecollections
fine_res_tiles = np.empty((x_tile_segments, y_tile_segments), ee.ImageCollection)
# array to track which dates of fine res images are used to get the MODIS images corresponding to the dates
fine_res_dates = np.empty((x_tile_segments, y_tile_segments), dtype = list)

for col in range(x_tile_segments):
    for row in range(y_tile_segments):
        #get a collection for all images within the date range, clipped to region and filtered for cloud cover
        initial_collection = (ee.ImageCollection(image_sets[satellite_choice]['imagecollection_id'])
                        .filterBounds(tiles[col, row])
                        .filterDate(*date_range)
                        .sort(image_sets[satellite_choice]['time_field'])
                        .map(lambda image: image.clip(tiles[col, row]))
#                         .filterMetadata('CLOUD_COVER', 'less_than' , 1)
#                       add filter for cloud mask
                     )
#         debug
#         print('initialcollectiondates')
#         for i in range(len(initial_collection.getInfo()['features'])):
#             print(initial_collection.getInfo()['features'][i]['id'])
#             print(initial_collection.getInfo()['features'][i]['properties']['CLOUD_COVER'])
        
        date_filtered_collections = []
        fine_res_dates[col][row] = []
        
        for date in prediction_dates:
#           create a field calculating the distance of images from each prediction date, and find the lowest two for each date to use for PSRFM
            initial_collection = initial_collection.map(
                lambda image: image.set(f'dateDist{date}', 
                                        ee.Number(image.get('system:time_start'))
                                        .subtract(ee.Date.millis(ee.Date(date)))
#                                         .abs()
                                       ))
#             closest_two_list = initial_collection.sort(f'dateDist{date}').toList(initial_collection.size())
            
            first_before = initial_collection.filterMetadata(f'dateDist{date}', 'less_than', 0).sort(f'dateDist{date}', False).first()
            first_after = initial_collection.filterMetadata(f'dateDist{date}', 'greater_than', 0).sort(f'dateDist{date}', True).first()
                     
# exception? following lines might fail if no valid images within date range: 
#             second_closest_image_dist = closest_two_list.get(1).getInfo()['properties'][f'dateDist{date}']
            closest_two_list = ee.ImageCollection([first_before, first_after])
#     closest_two_list.filter(
#                 ee.Filter.lessThanOrEquals(f'dateDist{date}', second_closest_image_dist))
#           Insert the dates selected for PSRFM to track which MODIS images to extract later
            first_image_date = first_before.getInfo()['properties']['system:time_start']
#     ee.Date(
#                 closest_two_list.get(0).getInfo()['properties']['system:time_start'])
            second_image_date = first_after.getInfo()['properties']['system:time_start']
#     ee.Date(
#                 closest_two_list.get(1).getInfo()['properties']['system:time_start'])
            fine_res_dates[col][row].append(first_image_date )
            fine_res_dates[col][row].append(second_image_date)     
#           Convert the two closest images into an imagcollecion and add to the relevant tile
            date_filtered_collections.append(ee.ImageCollection(closest_two_list))
        
        selected_date_collection = date_filtered_collections[0]
#         print(selected_date_collection.getInfo())
        for collection in range(len(date_filtered_collections) - 1):
#             embed()
            selected_date_collection.combine(selected_date_collection, date_filtered_collections[collection + 1])
        fine_res_tiles[col, row] = selected_date_collection
#         print(selected_date_collection.getInfo())

# coarse_res_tiles = np.empty((x_tile_segments, y_tile_segments), ee.ImageCollection)
# for col in range(x_tile_segments):
#     for row in range(y_tile_segments):
#         collection = (ee.ImageCollection(image_sets['modis']['imagecollection_id'])
#                         .filterBounds(tiles[col, row])
#                         .filterDate(*date_range)
#                         .sort(image_sets['modis']['time_field'])
#                         .map(lambda image: image.clip(tiles[col, row])))
#         coarse_res_tiles[col, row] = collection

pprint(len(fine_res_tiles[0,0].getInfo()['features']))

2


In [15]:
# export imagecollection tile array function
def export_tile_array(tile_array):
    for col in range(len(tile_array)):
        for row in range(len(tile_array[col])):
            for elem in range(ee.ImageCollection.size(tile_array[col, row]).getInfo()):
                print(type(tile_array[col, row]))
#                 task = ee.batch.Export.image.toDrive(
#                                         image = tile_array[col, row].get(elem).toInt16(),
#                                         region = tile_array[col, row].get(elem).getInfo()['coordinates'],
#                                         crs = 'EPSG:32653',
#                                         description = 'testimagepyexport')
#             task.start()

# export_tile_array(coarse_res_tiles)
# export_tile_array(fine_res_tiles)

In [ ]:
test_imgs = np.empty((x_tile_segments, y_tile_segments), ee.Image)
for col in range(x_tile_segments):
    for row in range(y_tile_segments):
        test_imgs[col, row] = ee.Image(fine_res_tiles[col, row].first())

# test_imgs

In [ ]:
mapIdDicts = np.empty((x_tile_segments, y_tile_segments), dict)
for col in range(x_tile_segments):
    for row in range(y_tile_segments):
        mapIdDicts[col, row] = (test_imgs[col, row]).getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 93, 'max': 1801})
# mapIdDicts

In [ ]:
xloc = west_long
yloc = north_lat 

map = folium.Map(location=[xloc, yloc])
for col in range(x_tile_segments):
    for row in range(y_tile_segments):
        folium.TileLayer(
            tiles = mapIdDicts[col, row]['tile_fetcher'].url_format,
            attr='Map Data &copy; <a href=https://earthengine.google.com/>Google Earth Engine</a>',
            overlay=True,
            name=f'img{col}, {row}'
        ).add_to(map)
map.add_child(folium.LayerControl())
map